<a href="https://www.kaggle.com/code/katzurasharma/xception-mod-output?scriptVersionId=114546167" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Importing Essentials

In [ ]:
import tensorflow as tf

In [ ]:
import glob, numpy as np, matplotlib.pyplot as plt, cv2.cv2 as cv2, seaborn as sns
import tensorflow.keras.callbacks as cb
from tensorflow.keras.applications import ResNet50, DenseNet121, NASNetMobile, Xception
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import glob

In [ ]:
"""data augmentation using keras"""
augmenter = ImageDataGenerator(rotation_range=90, horizontal_flip=True, vertical_flip=True, rescale=None)


"""model callbacks"""
checkpoint = cb.ModelCheckpoint(filepath='./checkpoints/densenet121/eps={epoch:03d}_valLoss={val_loss:.4f}.hdf5',
                                monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
cb_list = [checkpoint]

# Data Preperation

In [ ]:
SIZE=96
batch_size=32

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
                                   
train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/3classdata/covid19/Covid-19 Train/', 
        target_size=(SIZE, SIZE),
        batch_size=batch_size,
        classes = ['Negative','Positive'],
        color_mode='rgb', 
        class_mode='binary')  

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
        '/kaggle/input/3classdata/covid19/Covid-19 Valid/',  
        target_size=(SIZE, SIZE),
        batch_size=batch_size,
        classes = ['Negative','Positive'],
        color_mode='rgb',
        class_mode='binary')


## Training

In [ ]:
### Using pretrained Xception model
Model_Xcep = tf.keras.applications.Xception(include_top=False, weights='imagenet', input_shape=(SIZE, SIZE, 3))
Model_VGG = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(SIZE, SIZE, 3))

for layer in Model_Xcep.layers:
	layer.trainable = False
    
for layer in Model_VGG.layers:
	layer.trainable = False
    
Model_Xcep.summary()


# Modifying the output

In [ ]:
###### modified the output

model = tf.keras.Sequential([
    Model_Xcep, 
    tf.keras.layers.GlobalAveragePooling2D(), 
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Dropout(0.2), 
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model = tf.keras.Sequential([
    Model_VGG, 
    tf.keras.layers.GlobalAveragePooling2D(), 
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Dropout(0.2), 
    tf.keras.layers.Dense(1, activation='sigmoid')
])
    
    
#model = tf.keras.models.load_model ("Modified_Xception1.h5")    
model.summary()  

In [ ]:
import keras

## Compile and Fit

In [ ]:
# Adding callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("Modified_VGG1.h5", save_best_only=True, verbose = 0),
    tf.keras.callbacks.EarlyStopping(patience=4, monitor='val_accuracy', verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

# Compiling the model
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              loss = 'binary_crossentropy',
              metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),'accuracy'])

#### TRainging the model
history = model.fit(train_generator, 
                    validation_data=val_generator, epochs=15, 
                    callbacks=[callbacks])

In [ ]:
history.history

In [ ]:
model.save("/kaggle/working/Modified_Xception.h5")
model.save("/kaggle/working/Modified_VGG.h5")

# Metrics

In [ ]:
## Accuracy & Loss
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()



In [ ]:
### Precision, Recall, Macro F1 score
val_recall = history.history['val_recall']
print(val_recall)
avg_recall = np.mean(val_recall)
avg_recall

val_precision = history.history['val_precision']
avg_precision = np.mean(val_precision)
avg_precision

Train_accuracy = history.history['accuracy']

epochs = range(1, len(Train_accuracy)+1)
plt.figure(figsize=(12,6))
plt.plot(epochs, val_recall, 'g', label='Validation Recall')
plt.plot(epochs, val_precision, 'b', label='Validation Prcision')
plt.title('Validation recall and Validation Percision')
plt.xlabel('Epochs')
plt.ylabel('Recall and Precision')
plt.legend()
plt.ylim(0,1)

plt.show()




In [ ]:
###### The macro F1 score on the validation sex (0.78232)
Macro_F1score = (2*avg_precision*avg_recall)/ (avg_precision + avg_recall)
Macro_F1score

In [ ]:
valid_path='/kaggle/input/3classdata/covid19/Covid-19 Valid'
train_path='/kaggle/input/3classdata/covid19/Covid-19 Train'
test_path='/kaggle/input/3classdata/covid19/Covid-19 Test'

In [ ]:
train_image_files=glob.glob(train_path +'/*/*.jp*g')
valid_image_files=glob.glob(valid_path+'/*/*.jp*g')


In [ ]:
def get_confusion_matrix(data_path,N):
    #we need to see the data in the same order
    #for both predictions and targets
    print("Generating confusion matrix",N)
    predictions=[]
    targets=[]
    i=0
    for x,y in val_datagen.flow_from_directory(data_path,target_size=[SIZE,SIZE],shuffle=True):
        i+=1
        if i%50==0:
            print(i)
        p=model.predict(x)
        p=np.argmax(p,axis=1)
        y=np.argmax(y,axis=1)
        predictions=np.concatenate((predictions,p))
        targets=np.concatenate((targets,y))
        if len(targets)>N:
            break
    cm=confusion_matrix(targets,predictions)
    return cm

In [ ]:
cm=get_confusion_matrix(valid_path,len(train_image_files))
print(cm)

In [ ]:
valid_cm=get_confusion_matrix(valid_path,len(valid_image_files))
print(valid_cm)

In [ ]:
import itertools
def plot_confusion_matrix(cm,classes,normalize=False,title='Confusion matrix',
                         cmap=plt.cm.Blues):
    if normalize:
        cm=cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
        print("Normalize confusion matrix")
    else:
        print("Confusion matrix,without normalization")
    print(cm)
    
    plt.imshow(cm,interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks=np.arange(len(classes))
    plt.xticks(tick_marks,classes,rotation=45)
    plt.yticks(tick_marks,classes)
    
    fmt=".2f" if normalize else 'd'
    thresh= cm.max()/2.
    for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
        plt.text(j,i,format(cm[i,j],fmt),
        horizontalalignment="center",
        color='white' if cm[i,j]> thresh else 'black')
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
labels=['Negative','Positive']
plot_confusion_matrix(cm,labels,title='Train confusion matrix')

In [ ]:
plot_confusion_matrix(valid_cm,labels,title='Validation confusion matrix')

In [ ]:
np.trace(cm) /cm.sum()

In [ ]:
np.trace(valid_cm)/ valid_cm.sum()

In [ ]:
# # ============ Load Checkpoint ============
# model = keras.models.load_model("/kaggle/working/Modified_VGG.h5")
#  # get weights
# modelWeights = model.get_weights()
#  # get optimizer state as it was on last epoch
# modelOptimizer = model.optimizer

#  # ============ Compile Model ============
#  # redefine architecture (newModel=models.Sequential(), etc.)
# newModel= redefine_your_model_architecture()
#  #$ newModel= previous model architecture
#  # compile
# newModel.compile(optimizer=modelOptimizer,
#                   loss = 'binary_crossentropy',
#                   metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),'accuracy'])
#  # set trained weights
# newModel.set_weights(modelWeights)